In [1]:
import ROOT as rt
import uproot
# from root_numpy import root2array, tree2array
#from root_pandas import read_root
import h5py 

import numpy as np
import numpy.lib.recfunctions as nlr
import pandas as pd
import os, sys
from matplotlib import pyplot as plt
import math
import json

import gpustat
gpustat.print_gpustat()
os.environ['CUDA_VISIBLE_DEVICES']='3'

Welcome to JupyROOT 6.22/02
gpu-ibanks-3.hep.caltech.edu  Thu Sep 23 12:21:25 2021  470.57.02
[0] NVIDIA GeForce GTX 1080 | 73'C, 100 % |  4806 /  8119 MB | 3042(737M) 3054(4049M)
[1] NVIDIA GeForce GTX 1080 | 60'C,   0 % |  7511 /  8119 MB | 3054(7507M)
[2] NVIDIA GeForce GTX 1080 | 31'C,   0 % |     4 /  8119 MB |
[3] NVIDIA GeForce GTX 1080 | 50'C,   0 % |  7929 /  8119 MB | 3054(7925M)
[4] NVIDIA GeForce GTX 1080 | 28'C,   0 % |     4 /  8119 MB |
[5] NVIDIA GeForce GTX 1080 | 57'C,   0 % |  8008 /  8119 MB | 3012(723M) 3054(7281M)
[6] NVIDIA GeForce GTX 1080 | 56'C,   0 % |  7499 /  8119 MB | 3054(7495M)
[7] NVIDIA GeForce GTX 1080 | 71'C,  55 % |  4077 /  8119 MB | 3054(4057M)


In [7]:
def samp_to_df(samp_name):
    file_name = f'ML/DNN_Trees/combine_sequential_DNN/{samp_name}_combine_seqDNN.root'
    samp_file = uproot.open(file_name)
    samp_array = samp_file['tree'].arrays()
    samp_df = pd.DataFrame(samp_array)
    return samp_df

In [9]:
samples = ['sig', 'VBFHToGG', 'VHToGG', 'VBFHToGG', 'ttHToGG', 'GluGluHtoGG', 'DiPhotonJetsBox']
dfs = []
for samp in samples:
    dfs.append(samp_to_df(samp))

In [13]:
cat1_dfs = []
cat2_dfs = []
for df in dfs:
    c1_df = df[df[b'DNN_score'] >= 0.5]
    c2_df = df[df[b'DNN_score'] < 0.5]
    cat1_dfs.append(c1_df)
    cat2_dfs.append(c2_df)

In [17]:
for i in range(len(samples)):
    print(samples[i])
    c1_yield = np.sum(cat1_dfs[i].loc[:,b'genweight_scale'].values)
    c2_yield = np.sum(cat2_dfs[i].loc[:, b'genweight_scale'].values)
    print(f'Cat 1 Yield: {c1_yield}')
    print(f'Cat 2 Yield: {c2_yield}')
    print('\n')

sig
Cat 1 Yield: 1.9093897160832838
Cat 2 Yield: 0.2509009816880644


VBFHToGG
Cat 1 Yield: 2.2269970517374498
Cat 2 Yield: 5.859881736967286


VHToGG
Cat 1 Yield: 0.031141655074769098
Cat 2 Yield: 0.01577823321231607


VBFHToGG
Cat 1 Yield: 2.2269970517374498
Cat 2 Yield: 5.859881736967286


ttHToGG
Cat 1 Yield: 10.346382676815402
Cat 2 Yield: 66.83223159761457


GluGluHtoGG
Cat 1 Yield: 3.181379836146298
Cat 2 Yield: 30.18129993394717


DiPhotonJetsBox
Cat 1 Yield: 5485.605459307579
Cat 2 Yield: 112147.5713421699


